# Feature Engineering

In [1]:
import pandas as pd
import psycopg2
import sqlalchemy
import matplotlib
from sqlalchemy import create_engine

Establish connection with database

In [2]:
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'localhost' 
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' 
POSTGRES_PASSWORD = 'postgres' 
POSTGRES_DBNAME = 'greyhounds'

postgres_str = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}"

# Create the connection
cnx = create_engine(postgres_str)

cnx_str = "dbname='greyhounds' user='postgres' host='localhost' password='postgres'"

In [7]:
def long_to_wide(df, index, columns, values):
    """ Takes a dataframe that is in long format and converts it to wide format"""

    wide =  df.dropna(axis = 0, how = 'all').pivot(index = index, columns = columns, values = values)

    if isinstance(values, str):
        wide.columns = [values + '_' + str(colname) for colname in wide.columns]
    else:
        wide.columns = [' '.join(str(col)).strip().replace("(","").replace("'","").replace(",","_").replace(")","").replace(" ","") for col in wide.columns.values]
    return wide

In [26]:
def features_long_fn():
    '''
    Identifies the fastest time recorded in the last 25 days for each dog
    and stores result in min_time_long table. We also record the number
    of qualifying races
    Schema
    race_id min_time 
    Arguments
        see above
    Returns:
        None
    '''
    try:
        msg = 'Attempting to create the features_long table'
        print(msg)
    
        # Connect to database
        connect_str = "dbname='greyhounds' user='postgres' host='localhost' password='postgres'"
        conn_psql = psycopg2.connect(connect_str)
        cursor = conn_psql.cursor()

        # Create table
        cursor.execute("""
            DROP TABLE IF EXISTS temp_features;
            CREATE TABLE temp_features AS
            SELECT race_id, dog_id, box, min(previous_time) as min_time, 
                                         avg(previous_time) as avg_time,
                                         min(previous_stime) as min_stime,
                                         avg(previous_stime) as avg_stime,
                                         avg(previous_fin) as avg_fin,
                                         count(*) as no_qual_races
            FROM
            (SELECT 
                p1.box,
                p1.race_id,
                p1.dog_id,
                p1.fin,
                p2.race_id as previous_race_id,
                p2.time as previous_time,
                p2.stime as previous_stime,
                CAST(p2.fin AS float) as previous_fin
            FROM positions p1
            INNER JOIN positions p2
                ON p1.dog_id = p2.dog_id AND
                p1.race_id <> p2.race_id
            LEFT JOIN races r1
                ON p1.race_id = r1.race_id
            LEFT JOIN races r2
                ON p2.race_id = r2.race_id
            WHERE 
                r1.stadium_id = r2.stadium_id AND
                r1.distance = r2.distance AND
                r1.date_time > r2.date_time AND
                r1.date_time < r2.date_time + interval '25 days'
                ) subquery
            GROUP BY race_id, dog_id, box, fin
            ORDER BY race_id desc, box desc
            """)
        cursor.execute("""
            DROP TABLE IF EXISTS features_long;
            CREATE TABLE features_long AS
            SELECT p.race_id, p.dog_id, p.fin, p.box, mt.min_time, mt.avg_time, mt.min_stime, mt.avg_stime, mt.avg_fin, 
                CASE WHEN mt.no_qual_races IS NULL THEN 0
                ELSE mt.no_qual_races
                END no_qual_races
            FROM positions p
            LEFT JOIN temp_features mt ON
                mt.race_id = p.race_id AND
                mt.dog_id = p.dog_id
            ORDER BY p.race_id desc, p.box desc;
            DROP TABLE temp_features;
            """)
        conn_psql.commit()
        msg = "The features_long table has been created"
        print(msg)
        
    except (Exception, psycopg2.DatabaseError) as error:
        msg = "ERROR occured whilst creating the features_long table"
        print(msg)
        print(error)

    finally:
        if(conn_psql):
            cursor.close()
            conn_psql.close()

In [27]:
features_long_fn()

Attempting to create the features_long table
The features_long table has been created


In [28]:
features_long = pd.read_sql_query('''SELECT * FROM features_long''', cnx)
features_long[features_long['no_qual_races']>4]

,race_id,dog_id,fin,box,min_time,avg_time,min_stime,avg_stime,avg_fin,no_qual_races
2038,4621462,2383527,5,5,29.28,29.522,4.49,4.522,3.2,5
3563,4611797,2393673,3,6,29.25,29.502,4.43,4.490,3.4,5
3812,4610544,2395097,2,2,29.58,29.970,4.65,4.722,3.0,5
3977,4608865,2393673,5,5,29.21,29.430,4.43,4.474,2.6,5
4087,4608448,2392571,2,3,29.10,29.592,4.53,4.588,4.2,5
...,...,...,...,...,...,...,...,...,...,...
322258,85457,294810,2,4,29.18,29.472,4.33,4.474,3.4,5
322278,85454,325943,4,2,29.94,30.250,4.47,4.498,2.4,5
322323,84674,411653,5,4,29.51,29.690,4.41,4.556,2.8,5
322555,84280,411653,4,6,29.40,29.688,4.51,4.598,3.4,5


In [29]:
min_time = long_to_wide(features_long, 'race_id', 'box', 'min_time')
min_time

,min_time_1,min_time_2,min_time_3,min_time_4,min_time_5,min_time_6
race_id,,,,,,
16022,53.20,NaN,53.17,NaN,53.82,53.18
16809,NaN,NaN,NaN,29.57,NaN,NaN
16895,NaN,NaN,29.56,28.65,29.04,NaN
16896,NaN,29.23,29.68,29.43,NaN,29.55
16897,29.42,28.84,29.15,29.51,29.72,NaN
...,...,...,...,...,...,...
4642020,29.48,29.68,29.95,29.50,29.19,29.58
4642021,29.96,29.39,29.33,29.56,29.63,29.80
4642022,29.23,29.20,29.80,29.29,29.24,29.33


In [30]:
no_qual_races = long_to_wide(features_long, 'race_id', 'box', 'no_qual_races')
no_qual_races

,no_qual_races_1,no_qual_races_2,no_qual_races_3,no_qual_races_4,no_qual_races_5,no_qual_races_6
race_id,,,,,,
16022,1.0,0.0,1.0,0.0,1.0,1.0
16809,0.0,0.0,0.0,1.0,0.0,0.0
16895,0.0,0.0,2.0,3.0,1.0,0.0
16896,0.0,1.0,1.0,2.0,0.0,1.0
16897,1.0,2.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...
4642020,2.0,1.0,1.0,2.0,3.0,3.0
4642021,2.0,2.0,3.0,3.0,3.0,2.0
4642022,3.0,2.0,1.0,2.0,1.0,2.0


In [31]:
avg_time = long_to_wide(features_long, 'race_id', 'box', 'avg_time')
min_stime = long_to_wide(features_long, 'race_id', 'box', 'min_stime')
avg_stime = long_to_wide(features_long, 'race_id', 'box', 'avg_stime')
avg_fin = long_to_wide(features_long, 'race_id', 'box', 'avg_fin')

In [32]:
avg_fin

,avg_fin_1,avg_fin_2,avg_fin_3,avg_fin_4,avg_fin_5,avg_fin_6
race_id,,,,,,
16022,3.000000,NaN,1.000000,NaN,6.000000,2.000000
16809,NaN,NaN,NaN,4.000000,NaN,NaN
16895,NaN,NaN,3.500000,1.666667,1.000000,NaN
16896,NaN,1.0,2.000000,4.000000,NaN,2.000000
16897,3.000000,1.0,2.000000,3.000000,5.000000,NaN
...,...,...,...,...,...,...
4642020,4.500000,1.0,4.000000,3.500000,3.666667,2.666667
4642021,4.500000,4.0,2.333333,4.000000,3.333333,4.500000
4642022,3.666667,5.0,6.000000,6.000000,5.000000,4.500000
